In [14]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time


In [15]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
endpoint = "https://lab3-albertregus.cognitiveservices.azure.com/"
vision_key = "570a3afbac67498ab7a45c74ec317fa9"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(vision_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

# Problema 1

## 1. a) calitatea procesului de recunoastere a textului, atat la nivel de caracter, cat si la nivel de cuvant prin folosirea unei metrici de distanta

In [18]:
# img = open("test1.png", "rb")
# img = open("images/test2.jpeg", "rb")
img = open("images/firefox.png", "rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            # print(line.text)
            result.append(line.text)

print(result)

['Firefox']


In [19]:
# groundTruth = ["Google Cloud", "Platform"]
# groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]
groundTruth = ["Firefox"]

noOfCorrectLines = sum(i == j for i, j in zip(result, groundTruth))
print(noOfCorrectLines)

1


In [20]:
#original_text = "Success in rezolvarea tEMELOR la LABORA toarele de Inteligenta Artificiala!";

original_text = "Firefox"

- calitatea procesului de recunoastere a textului folosind distanta Levenshtein

In [21]:
import editdistance

def ocr_quality(original_text, ocr_output):
    distance = editdistance.eval(original_text, ocr_output)
    return distance

ocr_text = ' '.join(result)
print(ocr_text)

distance = ocr_quality(original_text, ocr_text)
print("Distanta Levenshtein: ", distance)

Firefox
Distanta Levenshtein:  0


- calitatea procesului de recunoastere a textului folosing distanta Hamming

In [22]:
def hamming_distance(s1, s2):

    # if len(s1) != len(s2):
    #     raise ValueError("Strings must be of equal length")
    
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

ocr_text = ' '.join(result)

distance = hamming_distance(original_text, ocr_text)
print("Distanta Hamming: ", distance)

Distanta Hamming:  0


- calitatea procesului de recunoastere a textului folosing distanta Jaro-Winkler <br>
0 - stringurile sunt total diferite <br>
1 - stringurile sunt la fel

In [23]:
import jellyfish

ocr_text = ' '.join(result)

jaro_winkler_distance = jellyfish.jaro_winkler_similarity(original_text, ocr_text)

print("Distanta Jaro-Winkler: ", jaro_winkler_distance)

Distanta Jaro-Winkler:  1.0


## 2) calitatea localizarii corecte a textului in imagine

In [1]:
import pytesseract
from pytesseract import Output
import cv2

In [7]:
image = cv2.imread("images/firefox.png")

if image is not None:
    cv2.imshow("Image", cv2.resize(image, (800, 600)))
    cv2.waitKey(0)  
    cv2.destroyAllWindows() 
else:
    print("Image not found or cannot be read.")

rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = pytesseract.image_to_data(rgb, output_type=Output.DICT)

print(results["text"], results["conf"])

['', '', '', '', '*', 'Firefox'] [-1, -1, -1, -1, 43, 96]


In [8]:
for i in range(0, len(results["text"])):
	
	x = results["left"][i]
	y = results["top"][i]
	w = results["width"][i]
	h = results["height"][i]

	text = results["text"][i]   # extrage textul 
	conf = int(results["conf"][i])  # nivelul de recunoastere

In [12]:
print("Confidence: {}".format(conf))
print("Text: {}".format(text))
print("")

text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Confidence: 96
Text: Firefox

